## Pretrain with MLM and Fine-tune with SequenceClassification

In [45]:
!pip install transformers
!pip install --upgrade torch torchvision torchaudio
!pip install wandb
!pip install scikit-learn
!pip install numpy

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [46]:
import os
import json
import torch
import random
import numpy as np
import torch.nn as nn
import wandb
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, AutoModelForMaskedLM, AutoModel, DataCollatorForLanguageModeling
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset
from imblearn.over_sampling import RandomOverSampler
from torch.optim import Adam
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [47]:
wandb.login(key="9b49f600300d891290c544a9d0580ec7b7185a34")
wandb.init(project='zaloqa-bert', entity='hdghung2912')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Train Accuracy,▁▇█
Train Loss,█▂▁
Train Accuracy,0.99767
Train Loss,0.00988


In [48]:
with open('/kaggle/input/uit-visquad/visquad_train.json', 'r', encoding='utf-8') as f:
    input_data = json.load(f)

In [49]:
def is_answer_correct(context, answer):
    return answer in context

In [50]:
pretrain_data = []
for item in input_data['data']:
    for paragraph in item['paragraphs']:
        context = paragraph.get('context', "")
        for qa in paragraph['qas']:
            question = qa.get('question', "")
            if 'answers' in qa:
                for answer in qa['answers']:
                    answer_text = answer.get('text', "")
                    label = is_answer_correct(context, answer_text)
                    pretrain_data.append({
                        "question": question,
                        "text": context,
                        "label": label
                    })
            if 'plausible_answers' in qa:
                for answer in qa['plausible_answers']:
                    answer_text = answer.get('text', "")
                    label = False  # Plausible answers are considered as false
                    pretrain_data.append({
                        "question": question,
                        "text": context,
                        "label": label
                    })


In [51]:
for i in range(min(5, len(pretrain_data))):
    print(json.dumps(pretrain_data[i], ensure_ascii=False, indent=4))

{
    "question": "Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?",
    "text": "Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).",
    "label": true
}
{
    "question": "Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà nước Cộng hòa Xã hội chủ nghĩa Việt Nam?",
    "text": "Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1

In [52]:
pretrain_file_path = '/kaggle/working/visquad_true_false.json'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(os.path.dirname(pretrain_file_path), exist_ok=True)

# Lưu dữ liệu mới vào file JSON
with open(pretrain_file_path, 'w', encoding='utf-8') as f:
    f.write(json.dumps(pretrain_data, ensure_ascii=False, indent=4))

In [53]:
# Load the data from train.json
with open('/kaggle/input/traintest/train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Split the data into train and test sets (85% train, 15% test)
train_data, test_data = train_test_split(data, test_size=0.15, random_state=42)

# Save the train and test data to separate files
with open('train_split.json', 'w', encoding='utf-8') as train_file:
    json.dump(train_data, train_file, ensure_ascii=False, indent=4)

with open('test_split.json', 'w', encoding='utf-8') as test_file:
    json.dump(test_data, test_file, ensure_ascii=False, indent=4)


In [54]:
# Đọc dữ liệu từ tập tin train_split.json vào biến train_data
with open('train_split.json', 'r', encoding='utf-8') as train_file:
    train_data = json.load(train_file)

# Đọc dữ liệu từ tập tin test_split.json vào biến test_data
with open('test_split.json', 'r', encoding='utf-8') as test_file:
    test_data = json.load(test_file)

In [55]:
pretrain_model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [56]:
class MLMPretrainDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data[idx]["question"]
        text = self.data[idx]["text"]
        input_text = question + " " + text
        encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"  # Sử dụng return_tensors="pt" để trả về tensor
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        
        return {"input_ids": input_ids, "attention_mask": attention_mask}


In [57]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
dataset = MLMPretrainDataset(pretrain_data, tokenizer, max_length=256)

# Data collator để chuẩn bị batch
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

dataloader = DataLoader(dataset, batch_size=16, collate_fn=data_collator)

In [58]:
for i, batch in enumerate(dataloader):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    print(f"Batch {i + 1}:")
    print("Input IDs:", input_ids)
    print("Attention Mask:", attention_mask)
    if i == 2:  # Print the first 3 batches
        break


Batch 1:
Input IDs: tensor([[    0,  4473,   328,  ...,     1,     1,     1],
        [    0, 49517,  2965,  ...,     1,     1,     1],
        [    0, 26268,   687,  ...,     1,     1,     1],
        ...,
        [    0,  6713,  2965,  ...,     1,     1,     1],
        [    0, 25254,   326,  ...,     1,     1,     1],
        [    0,   680,  5444,  ...,     1,     1,     1]])
Attention Mask: tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
Batch 2:
Input IDs: tensor([[    0,   268, 13466,  ...,     1,     1,     1],
        [    0, 24664,  6035,  ...,     1,     1,     1],
        [    0,   382,   423,  ...,     1,     1,     1],
        ...,
        [    0,  7657,  9901,  ...,     1,     1,     1],
        [    0,   146,  1941,  ...,     1,     1,     1],
        [    0,  6081,   212,  ...,     1,     1,     1]])
At

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pretrain_model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [60]:
optimizer = Adam(pretrain_model.parameters(), lr=5e-5)
epochs = 5
total_steps = len(dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [61]:
def pretrain(model, dataloader, optimizer, device, epochs, checkpoint_dir):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for step, batch in enumerate(tqdm(dataloader, desc=f"Pretrain Epoch {epoch + 1}/{epochs}")):
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        avg_train_loss = total_loss / len(dataloader)
        print(f"Pretrain Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}")
        
        wandb.log({"pretrain_batch_loss": loss.item()})
        
        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), checkpoint_path)

In [62]:
checkpoint_dir = './checkpoints'
#pretrain(pretrain_model, dataloader, optimizer, device, epochs, checkpoint_dir)

In [63]:
pretrain_checkpoint_dir = './pretrain_checkpoint'
os.makedirs(pretrain_checkpoint_dir, exist_ok=True)
pretrain_model.save_pretrained(pretrain_checkpoint_dir)

In [64]:
def preprocess_data(data, tokenizer):
    max_question_length = 64
    max_text_length = 192
    input_ids = []
    attention_masks = []
    labels = []

    for sample in data:
        question = sample['question']
        text = sample['text']
        label = sample['label']
        
        encoded_question = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            truncation=True,
            max_length=max_question_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        encoded_text = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=max_text_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        combined_input_ids = torch.cat((encoded_question['input_ids'], encoded_text['input_ids']), dim=1)
        combined_attention_mask = torch.cat((encoded_question['attention_mask'], encoded_text['attention_mask']), dim=1)

        input_ids.append(combined_input_ids)
        attention_masks.append(combined_attention_mask)
        labels.append(1 if label else 0)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    print(f"Pretrained input_ids shape: {input_ids.shape}")
    print(f"Pretrained attention_masks shape: {attention_masks.shape}")
    print(f"Pretrained labels shape: {labels.shape}")

    return input_ids, attention_masks, labels

In [65]:
X = [{'question': sample['question'], 'text': sample['text']} for sample in data]
y = [sample['label'] for sample in data]

# Chia dữ liệu thành tập huấn luyện và tập validation (85-15)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

# Tạo một RandomOverSampler
ros = RandomOverSampler(random_state=0)

# Resample tập huấn luyện
X_train_array = np.array([[sample['question'], sample['text']] for sample in X_train])
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_array, y_train)

# Đảm bảo X_train_resampled và y_train_resampled có cùng số lượng mẫu
assert len(X_train_resampled) == len(y_train_resampled)

# In ra số lượng nhãn true và false trong tập huấn luyện sau khi resample
num_true_resampled_train = sum(1 for label in y_train_resampled if label)
num_false_resampled_train = len(y_train_resampled) - num_true_resampled_train
print("Resampled Train - Number of true labels:", num_true_resampled_train)
print("Resampled Train - Number of false labels:", num_false_resampled_train)

Resampled Train - Number of true labels: 10535
Resampled Train - Number of false labels: 10535


In [66]:
resampled_data = [{'question': X_train_resampled[i, 0], 'text': X_train_resampled[i, 1], 'label': y_train_resampled[i]} for i in range(len(X_train_resampled))]

In [67]:
for i in range(5):
    print(f'Question: {X_train_resampled[i, 0]}')
    print(f'Text: {X_train_resampled[i, 1]}')
    print(f'Label: {y_train_resampled[i]}')
    print('---')

Question: Sông Nin đổ ra biển nào
Text: Sông Luỹ đổ ra biển ở cửa biển tại thị trấn Phan Rí
Label: False
---
Question: Tên gọi Nhật Bản nghĩa là gì
Text: Từ ghép này có nghĩa là " nguồn gốc của mặt trời " hoặc " nơi mặt trời mọc " ( từ quan điểm từ Trung Quốc , mặt trời mọc từ phía Nhật Bản ) ; nó là một nguồn cơ sở cho mô tả của phương Tây về Nhật Bản như là " Vùng đất Mặt trời mọc " ( " Land of the Rising Sun " ) .
Label: True
---
Question: Dầu mỏ có màu gì
Text: Loại dầu khoáng này là dầu trong suốt , không màu bao gồm chủ yếu là ankan và cycloankan , liên quan đến thạch dầu mỏ .
Label: False
---
Question: Huyện đảo Phú Quốc có diện tích bao nhiêu
Text: Hồ tiêu Phú Quốc là một loại gia vị được coi là đặc sản của huyện đảo Phú Quốc thuộc Tỉnh Kiên Giang , Việt Nam .
Label: False
---
Question: Cộng hoà Ireland có biên giới trên bộ với quốc gia nào
Text: Ireland là một quốc gia thành viên Liên minh châu Âu từ năm 1973 , song lựa chọn duy trì bên ngoài khu vực Schengen . Công dân Anh Qu

In [68]:
val_data = [{'question': sample['question'], 'text': sample['text'], 'label': label} for sample, label in zip(X_val, y_val)]

In [69]:
for i in range(5):
    print(f'Question: {val_data[i]["question"]}')
    print(f'Text: {val_data[i]["text"]}')
    print(f'Label: {val_data[i]["label"]}')
    print('---')

Question: Lê Lợi với Lê Lai có quan hệ gì
Text: Lê Thái Tổ ở ngôi được 5 năm thì qua đời vào ngày 22 tháng 8 âm lịch (7 tháng 9 dương lịch) năm Quý Sửu (1433), hưởng dương 49 tuổi. Vì nhớ công Lê Lai chết thay cho mình ở núi Chí Linh trước kia, ông dặn lại đời sau phải giỗ Lê Lai trước khi giỗ ông một ngày. Bởi thế đời sau truyền lại câu: "Hăm mốt Lê Lai, hăm hai Lê Lợi."
Label: False
---
Question: ai là phó bí thư hiện tại của Đà Nẵng
Text: Ngày 15 tháng 7 năm 2013 , Thành uỷ Đà Nẵng công bố quyết định số 7340 / QĐ-TU thành lập Ban Nội chính Thành uỷ Đà Nẵng . Ông Trần Thanh Vân , lúc này là Thành uỷ viên Thành uỷ Đà Nẵng , Viện trưởng Viện kiểm sát nhân dân thành phố Đà Nẵng , được điều động , bổ nhiệm làm Trưởng Ban Nội chính Thành uỷ Đà Nẵng nhiệm kì 5 năm , hai Phó ban là Nhật Thành , Phó Trưởng Ban thường trực Ban chỉ đạo Phòng chống tham nhũng thành phố Đà Nẵng và ông Phạm Hà Bắc , Phó Trưởng Ban chỉ đạo phòng chống tham nhũng thành phố Đà Nẵng .
Label: False
---
Question: Ai là

In [70]:
checkpoint_pretrain = '/kaggle/input/uit-mlm/transformers/uit-mlm/1/checkpoint-pretrained'

In [71]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_pretrain, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/uit-mlm/transformers/uit-mlm/1/checkpoint-pretrained and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [72]:
train_input_ids, train_attention_masks, train_labels = preprocess_data(resampled_data, tokenizer)
val_input_ids, val_attention_masks, val_labels = preprocess_data(val_data, tokenizer)

Pretrained input_ids shape: torch.Size([21070, 256])
Pretrained attention_masks shape: torch.Size([21070, 256])
Pretrained labels shape: torch.Size([21070])
Pretrained input_ids shape: torch.Size([2717, 256])
Pretrained attention_masks shape: torch.Size([2717, 256])
Pretrained labels shape: torch.Size([2717])


In [73]:
# Tạo TensorDataset cho tập huấn luyện
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)

# Tạo DataLoader cho tập huấn luyện
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)

# Tạo TensorDataset cho tập validation
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)

# Tạo DataLoader cho tập validation
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [74]:
for batch in train_dataloader:
    batch_input_ids, batch_attention_masks, batch_labels = batch
    print(f"Batch input_ids shape: {batch_input_ids.shape}")
    print(f"Batch attention_masks shape: {batch_attention_masks.shape}")
    print(f"Batch labels shape: {batch_labels.shape}")
    break

Batch input_ids shape: torch.Size([16, 256])
Batch attention_masks shape: torch.Size([16, 256])
Batch labels shape: torch.Size([16])


In [75]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [76]:
optimizer = Adam(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

In [77]:
epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [78]:
def train(model, train_dataloader, epochs, optimizer, scheduler, device, checkpoint_dir):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        total_loss = 0
        correct_preds = 0
        total_preds = 0
        # Sử dụng tqdm để hiển thị tiến trình huấn luyện
        for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
            batch_input_ids, batch_attention_masks, batch_labels = tuple(t.to(device) for t in batch)
            model.zero_grad()

            # Sử dụng AutoModelForMaskedLM để tạo ra một instance của mô hình cho mỗi loại dữ liệu
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks, labels=batch_labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_preds += torch.sum(preds == batch_labels).item()
            total_preds += len(batch_labels)

        avg_train_loss = total_loss / len(train_dataloader)
        train_accuracy = correct_preds / total_preds
        
        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
        
        wandb.log({"Train Loss": avg_train_loss, "Train Accuracy": train_accuracy})

            
        os.makedirs(checkpoint_dir, exist_ok=True)
        # Lưu checkpoint sau mỗi epoch
        checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch_{epoch + 1}.pt"
        torch.save(model.state_dict(), checkpoint_path)

In [79]:
checkpoint_path = './results/checkpoint'

In [80]:
checkpoint_dir = '/kaggle/input/uit-mlm/transformers/uit-mlm/2/checkpoint'

# Tìm checkpoint cuối cùng
checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("checkpoint_epoch_") and f.endswith(".pt")]
if not checkpoints:
    raise ValueError("No checkpoints found in directory.")
    
# Sắp xếp và lấy checkpoint có số thứ tự lớn nhất
checkpoints.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))
last_checkpoint_path = os.path.join(checkpoint_dir, checkpoints[-1])

# Nạp checkpoint vào mô hình
model.load_state_dict(torch.load(last_checkpoint_path, map_location=torch.device('cpu')))
print(f"Loaded checkpoint from {last_checkpoint_path}")

Loaded checkpoint from /kaggle/input/uit-mlm/transformers/uit-mlm/2/checkpoint/checkpoint_epoch_7.pt


In [81]:
train(model, train_dataloader, epochs, optimizer, scheduler, device, checkpoint_path)

Epoch 1/5


Training: 100%|██████████| 1317/1317 [14:39<00:00,  1.50it/s]


Epoch 1, Average Training Loss: 0.07666145806871184, Train Accuracy: 0.9832463217845278
Epoch 2/5


Training: 100%|██████████| 1317/1317 [14:38<00:00,  1.50it/s]


Epoch 2, Average Training Loss: 0.04442352545924977, Train Accuracy: 0.9906502135738016
Epoch 3/5


...

Epoch 3, Average Training Loss: 0.030552848703547703, Train Accuracy: 0.9937351684859991
Epoch 4/5


Training: 100%|██████████| 1317/1317 [14:37<00:00,  1.50it/s]


Epoch 4, Average Training Loss: 0.018740296039434456, Train Accuracy: 0.996203132415757
Epoch 5/5


Training: 100%|██████████| 1317/1317 [14:37<00:00,  1.50it/s]


Epoch 5, Average Training Loss: 0.006504928956115276, Train Accuracy: 0.9986236355007119


In [82]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in dataloader:            
            batch_input_ids, batch_attention_masks, batch_labels = tuple(t.to(device) for t in batch)

            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_masks)
            logits = outputs.logits
            predictions.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(batch_labels.tolist())

    predictions = np.array(predictions)
    true_labels = np.array(true_labels)

    f1 = f1_score(true_labels, predictions)
    cm = confusion_matrix(true_labels, predictions)

    return f1, cm

In [83]:
evaluate_model(model, val_dataloader, device)

(0.7944598337950138,
 array([[1629,  206],
        [ 165,  717]]))

In [84]:
model.save_pretrained(checkpoint_path)
tokenizer.save_pretrained(checkpoint_path)

('./results/checkpoint/tokenizer_config.json',
 './results/checkpoint/special_tokens_map.json',
 './results/checkpoint/vocab.txt',
 './results/checkpoint/bpe.codes',
 './results/checkpoint/added_tokens.json')